# Ghost Architect — Colab T4 Main Notebook

This is the single notebook to run full **Gemma-3 Trinity training** on Google Colab T4 (16GB).

## What this notebook does
1. Validates T4 runtime
2. Installs exact dependencies
3. Syncs project files
4. Writes full T4 training configuration
5. Launches training
6. Exports GGUF artifacts


## 1) Runtime Check (must be T4 GPU)

In [1]:
import torch

assert torch.cuda.is_available(), 'CUDA is not available. Set Runtime > GPU in Colab.'
gpu_name = torch.cuda.get_device_name(0)
gpu_mem_gb = torch.cuda.get_device_properties(0).total_memory / 1024**3
print(f'GPU: {gpu_name}')
print(f'VRAM: {gpu_mem_gb:.1f} GB')
if 'T4' not in gpu_name:
    print('Warning: This notebook is tuned for T4; adjust config if using a different GPU.')

!nvidia-smi


GPU: Tesla T4
VRAM: 14.6 GB
Tue Feb 24 17:24:18 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 580.82.07              Driver Version: 580.82.07      CUDA Version: 13.0     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   40C    P8             13W /   70W |       3MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-------------------

## 2) Install Dependencies

In [2]:
!pip install -q --retries 10 --timeout 120 --upgrade pip setuptools wheel "jedi>=0.16"
!pip install -q --retries 10 --timeout 120 "unsloth==2026.1.4"
!pip install -q --retries 10 --timeout 120 "trl>=0.18.2,<=0.24.0,!=0.19.0"
!pip install -q --retries 10 --timeout 120 peft accelerate bitsandbytes datasets numpy scipy tqdm
!pip install -q --retries 10 --timeout 120 "torch>=2.1.0" "transformers>=4.38.0"
!pip check || true

print('Dependencies installed. xformers is optional and intentionally not force-installed.')

No broken requirements found.
Dependencies installed. xformers is optional and intentionally not force-installed.
Dependencies installed. xformers is optional and intentionally not force-installed.


## 3) Mount Drive and Sync Repository

In [ ]:
from google.colab import drive
import os
import glob
import shutil

drive.mount('/content/drive')

PROJECT_DIR = '/content/ghost_architect_gemma3'
DRIVE_ROOT = '/content/drive/MyDrive'
DRIVE_COPY = f'{DRIVE_ROOT}/ghost_architect_gemma3'
REPO_URL = ''  # Optional: set your GitHub repo URL if Drive copy is unavailable

def copy_project(src_path: str):
    shutil.rmtree(PROJECT_DIR, ignore_errors=True)
    shutil.copytree(src_path, PROJECT_DIR)
    print(f'Copied project from: {src_path}')

if os.path.exists(f'{DRIVE_COPY}/src/train.py'):
    copy_project(DRIVE_COPY)
elif REPO_URL.strip():
    !rm -rf {PROJECT_DIR}
    !git clone {REPO_URL} {PROJECT_DIR}
else:
    candidates = sorted({
        path.rsplit('/src/train.py', 1)[0]
        for path in glob.glob(f'{DRIVE_ROOT}/**/src/train.py', recursive=True)
    })
    if len(candidates) == 1:
        copy_project(candidates[0])
    elif len(candidates) > 1:
        print('Multiple project candidates found in Drive:')
        for c in candidates[:20]:
            print(f' - {c}')
        raise RuntimeError('Set DRIVE_COPY to the correct path and rerun this cell.')
    elif os.path.exists(f'{PROJECT_DIR}/src/train.py'):
        print('Using existing project at /content/ghost_architect_gemma3')
    else:
        print('No project found in Drive. Top-level MyDrive entries:')
        for name in sorted(os.listdir(DRIVE_ROOT))[:50]:
            print(f' - {name}')
        raise FileNotFoundError(
            'Project not found. Upload ghost_architect_gemma3 to Drive, set DRIVE_COPY, or set REPO_URL.'
        )

%cd /content/ghost_architect_gemma3
assert os.path.exists('src/train.py'), 'src/train.py missing. Sync repo first.'


## 3.5) Environment Validation

In [ ]:
!python scripts/validate_environment.py

## 4) Full T4 Trinity Training Config

In [ ]:
config_yaml = '''
model_name: "unsloth/gemma-3-12b-it-bnb-4bit"
max_seq_length: 4096
load_in_4bit: true

lora:
  r: 64
  lora_alpha: 32
  target_modules:
    - q_proj
    - k_proj
    - v_proj
    - o_proj
    - gate_proj
    - up_proj
    - down_proj
  use_rslora: true
  use_dora: true
  lora_dropout: 0.1

training:
  per_device_train_batch_size: 1
  gradient_accumulation_steps: 4
  learning_rate: 2e-4
  max_steps: 60
  warmup_steps: 10
  logging_steps: 1
  save_steps: 20
  optimizer: "adamw_8bit"
  lr_scheduler_type: "cosine"

output:
  adapters_dir: "output/adapters"
  checkpoints_dir: "output/checkpoints"
  gguf_dir: "output/gguf"

oom_fallbacks:
  - {action: reduce_seq_len, value: 2048}
  - {action: reduce_rank, value: 32}
  - {action: disable_dora, value: false}
'''

os.makedirs('configs', exist_ok=True)
with open('configs/training_config_colab_t4.yaml', 'w') as f:
    f.write(config_yaml)

print('Wrote configs/training_config_colab_t4.yaml')

## 5) Validate Dataset

In [ ]:
import os, json

dataset_path = 'data/dataset.json'
assert os.path.exists(dataset_path), f'Missing dataset: {dataset_path}'

with open(dataset_path, 'r') as f:
    raw = f.read().strip()

if raw:
    _ = json.loads(raw)
    print('Dataset JSON is valid.')
else:
    print('Dataset file is empty. Populate data/dataset.json before training.')

## 6) Launch Full Training

In [ ]:
!python src/train.py --config configs/training_config_colab_t4.yaml --dataset data/dataset.json

## 7) Export to GGUF

In [ ]:
# Expected CLI in future implementation:
# !python src/export.py --adapter_dir output/adapters --output_dir output/gguf --quantization q4_k_m

print('Implement src/export.py next, then run the command above.')